![DSME-logo](./img/DSME_logo.png)

#  Reinforcement Learning and Learning-based Control

<p style="font-size:12pt";> 
<b> Prof. Dr. Sebastian Trimpe, Dr. Friedrich Solowjow </b><br>
<b> Institute for Data Science in Mechanical Engineering (DSME) </b><br>
<a href = "mailto:rllbc@dsme.rwth-aachen.de">rllbc@dsme.rwth-aachen.de</a><br>
</p>

---

# Comparing the abilities of TD(0) and constant-&alpha; MC

## Library Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import gymnasium as gym
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import custom_envs
import copy

## Initializations

In this example, we compare the prediction abilities of TD(0) and constant-&alpha; MC. To do this, we use both methods for learning state value functions based on a given policy and compare how the deviation to the true state value function changes over the training episodes. For the comparison, we use Frozenlake, with a simple map of the form:

<img src="./img/env.png" alt="Environment" width="400" height="400" />

The agent's goal is to achieve the gift. Upon reaching the gift, the agent receives a reward of "1"; on any other space, the agent receives a reward of "0".


### Obtaining Policy and State Value Function

To compare the two methods, we first need a policy, as well as the associated value function. Here, we use policy iteration to determine both. Dynamic Programming is particularly suitable for determining the state value function that we use for reference, since it can determine it stably with adjustable accuracy, specified in `update_threshold`. The details of the implementation of the agent that performs the policy iteration are assumed to be given here. For more details on Dynamic Programming, we refer to previous lectures.


In [ ]:
class MarkovDecisionProcess():
    def __init__(self, num_states, dynamics_fn):
        self.num_states = num_states
        self.actions_per_state = [] # list containing the actions available per state
        for state in dynamics_fn:
            actions = list(dynamics_fn[state].keys())
            self.actions_per_state.append(actions)
        self.P = dynamics_fn # P[s][a] represents a list of possible transitions and rewards given state s and a

class DP_Agent():
    def __init__(self, mdp, gamma=0.9, update_threshold=0.05):
        self.mdp = mdp # contains state transition function, num_states and actions
        self.update_threshold = update_threshold # stopping distance as criteria for stopping policy evaluation
        self.state_value_fn = np.zeros(self.mdp.num_states) # a table leading from state to value expectations
        self.policy = [[1 for _ in actions] for actions in self.mdp.actions_per_state]
        self.policy = [[action / len(actions) for action in actions] for actions in self.policy]
        self.gamma = gamma # discount rate for return

    def get_action(self, state):
        # Choose action based on the probabilities defined within the policy
        action = np.random.choice(np.flatnonzero(np.isclose(self.policy[state], max(self.policy[state]))))
        return action

    def get_action_value(self, state, action):
        # Calculate action value for state action pair based on the transition probabilities of the mdp
        action_value = 0
        for transition in self.mdp.P[state][action]:
            transition_prob = transition[0]  # prob of next state
            state_value_obs = transition[1]  # id of next state
            reward = transition[2]  # reward of next state
            action_value += transition_prob * (reward + self.gamma * self.state_value_fn[state_value_obs])
        return action_value

    def train(self):
        policy_stable = False
        while not policy_stable:
            # Use policy evaluation to update the value function
            self.policy_evaluation()
            # Use new value function to update the policy
            policy_stable = self.improve_policy() # if the policy stops changing, this ends the training process

    def policy_evaluation(self):
        # Policy evaluation for all states
        max_update = self.update_threshold + 1
        while max_update > self.update_threshold:
            max_update = 0
            for state in range(self.mdp.num_states):
                old_state_value = self.state_value_fn[state]
                new_state_value = 0
                actions = np.nonzero(self.policy[state])[0]
                for action in actions:
                    action_prob = self.policy[state][action]
                    new_state_value += action_prob * self.get_action_value(state, action)
                self.state_value_fn[state] = max(new_state_value, old_state_value)
                max_update = max(max_update, np.abs(old_state_value - self.state_value_fn[state]))

    def improve_policy(self):
        # Initiates improving the policy
        policy_stable = True
        # Cache current policy
        current_policy = self.policy
        # Create empty policy to store the best possible actions based on current value function
        best_policy = [[0 for _ in actions] for actions in self.mdp.actions_per_state]
        for state in range(self.mdp.num_states):
            # Calculate best possible policy based on current value function
            action_values = []
            for action in range(len(self.mdp.actions_per_state[state])):
                action_values.append(self.get_action_value(state, action))  # calculate q(s,a)
            action_value_max = max(action_values)
            best_actions = np.where(action_values == action_value_max)[0] # find indices where maximum value occurs
            for index in best_actions:
                best_policy[state][index] = 1
            best_policy[state] = [best_policy[state][action] / len(best_actions)
                                  for action in self.mdp.actions_per_state[state]]
            # If the current policy is not the best policy, update it
            if not np.array_equal(current_policy[state], best_policy[state]):
                policy_stable = False
                self.policy[state] = best_policy[state]
        return policy_stable

    def visualize(self):
        print('State value function of the agent:')
        x_axis = 4
        y_axis = 4
        vmin = 0
        vmax = 1
        X1 = np.reshape(self.state_value_fn, (x_axis, y_axis))
        fig, ax = plt.subplots(1, 1)
        cmap = cm.get_cmap("Blues_r")
        cmap.set_under("black")
        img = ax.imshow(X1, interpolation="nearest", vmin=vmin, vmax=vmax, cmap=cmap)
        ax.axis('off')
        for i in range(x_axis):
            for j in range(y_axis):
                ax.text(j, i, str(X1[i][j])[:4], fontsize=12, color='black', ha='center', va='center')
        plt.show()

    def render_policy(self):
        pol = np.copy(self.policy)
        print('Policy of the agent:')
        print('-----------------------------')
        out = '| '
        for i in range(self.mdp.num_states):
            token = ""
            if pol[i, 3] > 0:   # left
                token += "\u2190"
            if pol[i, 0] > 0:   # up
                token += "\u2191"
            if pol[i, 1] > 0:   # down
                token += "\u2193"
            if pol[i, 2] > 0:   # right
                token += "\u2192"
            if token == "":     # empty
                token += "  "

            if len(token) == 1:
                token += '  '
                token = ' ' + token
            elif len(token) == 2:
                token += ' '
                token = ' ' + token
            elif len(token) == 3:
                token += ' '

            out += token + ' | '
            if (i + 1) % 4 == 0:
                print(out)
                print('-----------------------------')
                out = '| '

We now create the map, the environment and the agent, let the agent train until the optimal policy and the corresponding value function is found and then extract the state value function and policy. Under `target_state_value_fn` the state value function determined via DP now serves as a reference for TD(0) and constant-&alpha; MC.

In [ ]:
# Set up map, environment, and agent
map = ["SFFF", "FFFF", "FFFF", "FFFG"]
env = gym.make('CustomFrozenLake-v1', render_mode=None, desc=map, is_slippery=False) # set render_mode=None for speeding up learning
env.reset()
gamma = 0.9
mdp = MarkovDecisionProcess(env.observation_space.n, env.unwrapped.P) # in our case contains dynamics function
agent = DP_Agent(mdp, update_threshold=0.00001)

# Train the agent.
agent.train()

# Extract the necessary functions for comparison
policy = agent.policy
target_state_value_fn = agent.state_value_fn

Below, we show both the state value function, and the resulting policy. The optimal policy in this environment always leads down-right and thus towards the gift. Since we use a discount factor &gamma; with the optimal policy, the state-value decreases from the terminal state to the initial state for each field by the factor &gamma;.

In [ ]:
# Show the policy
agent.render_policy()

# Show the value function
agent.visualize()

Now that the policy and the target value function are ready, we can turn our attention to the methods for determining the state value function.

We implement the two methods in a way that returns a measure of the progress of learning for each episode. We use the root mean squared (RMS) error between the learned value function and the target value function. The methods return an array containing the RMS error of all learning episodes.

### Constant-&alpha; MC

With constant alpha MC, the update of the state value function is determined by comparing the return of a state with its current state value. In case there are deviations, the difference multiplied by the factor &alpha; is added to the state value.
Updates are always performed after the end of the episode for all passed states. In this implementation, updates are performed for every visit, whereby each visit is also a first visit due to the optimal policy.

In [ ]:
def constant_alpha_MC(gamma, alpha, env, num_episodes, target_state_value_fn, policy):
    # Initialize state value function
    state_value_fn = np.zeros(mdp.num_states)
    rms_errors = [] 
    for i_episode in range(1, num_episodes + 1):
        # Generate an episode using the policy
        episode = []
        obs, info = env.reset()
        done = False
        while not done:
            action = np.random.choice(len(policy[obs]), p=policy[obs])
            next_obs, reward, done, truncated, info = env.step(action)
            episode.append((obs, reward))
            obs = next_obs
        #print('NEW EPISODE')
        episode = np.array(episode)
        episode_duration = len(episode[:,:1])
        # Calculate returns for the whole episode from the back to save memory and resources
        G = np.zeros([episode_duration, ])
        for i in range(episode_duration - 1, -1, -1):
            if i + 1 > episode_duration - 1:
                G[i] = episode[i][1] # Last step of the episode
            else:
                G[i] = episode[i][1] + gamma * G[i + 1] # Every other step
        # Update the policy with constant-alpha MC
        for index in range(episode_duration):
            obs = int(episode[index][0])
            #print('state:', obs)
            #print('old value fn', state_value_fn[obs])
            state_value_fn[obs] = state_value_fn[obs] + alpha * (G[index] - state_value_fn[obs])
            #print('new value fn', state_value_fn[obs])
        # Calculate root mean squared error for the episode
        rms_error = np.sqrt(np.mean(np.square(state_value_fn - target_state_value_fn)))
        rms_errors.append(rms_error)
    return rms_errors

### TD(0)

In TD methods, the updates for the state value function are performed using the TD error. This error is the deviation between the state value of a state and the immediate reward, plus the discounted state value of the subsequent state. As with constant-&alpha; MC, in case of deviation, the error is added to the state value of a state with a factor alpha, however, not at the end of the episode but immediately. For TD(0), it is sufficient to observe a state, its reward, and subsequent state to determine the updates. The opposite of this behavior is TD(1), where the entire episode would be used for the error, so that updates could only be made at the end of the episode.

In [ ]:
def td_0(gamma, alpha, env, num_episodes, target_state_value_fn, policy):
    # Initialize state value function
    state_value_fn = np.zeros(mdp.num_states)
    rms_errors = []
    for i in range(num_episodes):
        # Update the value function based on the policy
        obs, info = env.reset() # Initial state
        done = False
        #print('NEW EPISODE')
        while not done:
            action = np.random.choice(len(policy[obs]), p=policy[obs]) # Get action from policy
            next_obs, reward, done, truncated, info = env.step(action)
            #print('state:', obs)
            #print('old value fn', state_value_fn[obs])
            state_value_fn[obs] = state_value_fn[obs] + alpha * (reward + ( gamma * state_value_fn[next_obs] ) - state_value_fn[obs])
            #print('old value fn', state_value_fn[obs])
            obs = next_obs
        # Calculate root mean squared error for the episode
        rms_error = np.sqrt(np.mean(np.square(state_value_fn - target_state_value_fn)))
        rms_errors.append(rms_error)
    return rms_errors

## Learning the State Value Functions

To obtain meaningful values, we perform the learning of the state value functions for both methods with multiple values of &alpha;. In addition, we use multiple runs for each value of &alpha; and average their values for the episodes using the `data_generator` method.

In [ ]:
def data_generator(func, n_repeats, gamma, alpha, env, evaluation_episodes, target_state_value_fn, policy):
    data = []
    # Generate the data
    for i in range(n_repeats):
        data.append(func(gamma, alpha, env, evaluation_episodes, target_state_value_fn, policy))
    # Take the mean over the number of repeats
    mean_data = np.mean(data, axis=0)
    return mean_data

Now we use the `data_generator` to learn the state value functions for several values of &alpha; TD(0) and constant-&alpha; MC to get a plot of the mean RMS errors for each episode.

In [ ]:
n_repeats = 50
evaluation_episodes = 300

# Generate MC runs
mc_10 = data_generator(constant_alpha_MC, n_repeats, gamma, 0.1, env, evaluation_episodes, target_state_value_fn, policy)
mc_20 = data_generator(constant_alpha_MC, n_repeats, gamma, 0.2, env, evaluation_episodes, target_state_value_fn, policy)
mc_30 = data_generator(constant_alpha_MC, n_repeats, gamma, 0.3, env, evaluation_episodes, target_state_value_fn, policy)

# Generate TD runs
td_10 = data_generator(td_0, n_repeats, gamma, 0.1, env, evaluation_episodes, target_state_value_fn, policy)
td_20 = data_generator(td_0, n_repeats, gamma, 0.2, env, evaluation_episodes, target_state_value_fn, policy)
td_30 = data_generator(td_0, n_repeats, gamma, 0.3, env, evaluation_episodes, target_state_value_fn, policy)

## Comparing the Results

Below we plot the RMS errors over the episodes. The dashed graphs represent constant-&alpha; MC, the solid ones represent TD(0). 
Both methods succeed in approximating the target value function, which is reflected in the reduction of the RMS error to almost zero.

In [ ]:
# Display the data
fig, ax = plt.subplots(figsize=(10, 6))  # Create a figure and an axes.
ax.plot(range(evaluation_episodes), mc_10, label="constant-alpha MC - alpha=0.10", linestyle='dashed', color='blue')
ax.plot(range(evaluation_episodes), mc_20, label="constant-alpha MC - alpha=0.20", linestyle='dashed', color='red')
ax.plot(range(evaluation_episodes), mc_30, label="constant-alpha MC - alpha=0.30", linestyle='dashed', color='orange')
ax.plot(range(evaluation_episodes), td_10, label="TD(0) - alpha=0.10", color='blue')
ax.plot(range(evaluation_episodes), td_20, label="TD(0) - alpha=0.20", color='red')
ax.plot(range(evaluation_episodes), td_30, label="TD(0) - alpha=0.30", color='orange')
ax.grid(False)
ax.set_xlabel('Episodes')
ax.set_ylabel('Root Mean Squared Error')
ax.set_title('Root Mean Squared Error Averaged Over States')
ax.set_ylim(0, 0.8)
ax.set_xlim(0, evaluation_episodes)
ax.legend()
plt.show()